# Grad x Input Benchmark

In [1]:
import sys
sys.path.append("../")

import fastISM
from fastISM.models.basset import basset_model

from fastISM.models.factorized_basset import factorized_basset_model
from fastISM.models.bpnet import bpnet_model
import tensorflow as tf
import numpy as np
from importlib import reload
import time

In [2]:
reload(fastISM.flatten_model)
reload(fastISM.models)
reload(fastISM.ism_base)
reload(fastISM.change_range)
reload(fastISM.fast_ism_utils)
reload(fastISM)

<module 'fastISM' from '../fastISM/__init__.py'>

In [3]:
tf.__version__

'2.3.0'

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
device = 'GPU:0' if tf.config.experimental.list_physical_devices('GPU') else '/device:CPU:0'
device

'GPU:0'

## Benchmark

### Basset/Factorized Basset

In [29]:
BATCH_SIZES = [1,32,64,128,256]

In [32]:
# shap_values most likely internally creates a batch for each example
# thus time per 100 examples stays near constant with batch size

NUM_TO_AVG = 100

for model_type in [basset_model, factorized_basset_model]:
    for seqlen in [1000, 2000]:
        print("\n------------------")
        print("MODEL: {}".format(model_type))
        print("SEQLEN: {}".format(seqlen))
        model = model_type(seqlen=seqlen, num_outputs=1)
        
        # dry run 
        p = model(np.random.random((10,seqlen,4)))
        
        times = []
        per_100 = []
        for b in BATCH_SIZES:
            tot = 0
            for i in range(NUM_TO_AVG):
                x = np.random.random((b,seqlen,4))
                
                t = time.time()
                x = tf.constant(x)
                with tf.GradientTape() as tape:
                    tape.watch(x)
                    pred = model(x)
                g = (x*tape.gradient(pred, x)).numpy()
                
                tot+= time.time() - t
                
            times.append(tot/NUM_TO_AVG)
            per_100.append((times[-1]/b)*100)
            print("BATCH SIZE: {}\tTIME: {:.2f}\tPER 100: {:.2f}".format(b, times[-1], (times[-1]/b)*100))
        
        print("BEST PER 100: {:.2f}".format(min(per_100)))


------------------
MODEL: <function basset_model at 0x7f6cec170680>
SEQLEN: 1000
BATCH SIZE: 1	TIME: 0.03	PER 100: 2.87
BATCH SIZE: 32	TIME: 0.05	PER 100: 0.15
BATCH SIZE: 64	TIME: 0.06	PER 100: 0.09
BATCH SIZE: 128	TIME: 0.09	PER 100: 0.07
BATCH SIZE: 256	TIME: 0.17	PER 100: 0.07
BEST PER 100: 0.07

------------------
MODEL: <function basset_model at 0x7f6cec170680>
SEQLEN: 2000
BATCH SIZE: 1	TIME: 0.03	PER 100: 2.69
BATCH SIZE: 32	TIME: 0.06	PER 100: 0.20
BATCH SIZE: 64	TIME: 0.09	PER 100: 0.15
BATCH SIZE: 128	TIME: 0.17	PER 100: 0.14
BATCH SIZE: 256	TIME: 0.35	PER 100: 0.14
BEST PER 100: 0.14

------------------
MODEL: <function factorized_basset_model at 0x7f6cec170290>
SEQLEN: 1000
BATCH SIZE: 1	TIME: 0.06	PER 100: 6.46
BATCH SIZE: 32	TIME: 0.09	PER 100: 0.29
BATCH SIZE: 64	TIME: 0.12	PER 100: 0.19
BATCH SIZE: 128	TIME: 0.21	PER 100: 0.16
BATCH SIZE: 256	TIME: 0.39	PER 100: 0.15
BEST PER 100: 0.15

------------------
MODEL: <function factorized_basset_model at 0x7f6cec170290>
SEQ

### BPNet

In [6]:
BATCH_SIZES = [1, 32, 64, 128]

In [11]:
for seqlen in [1000, 2000]:
    print("\n------------------")
    print("SEQLEN: {}".format(seqlen))
    model = bpnet_model(seqlen=seqlen, num_dilated_convs=9)

    # run explainers for each position
    times = []
    per_100 = []

    # dry run 
    p = model(np.random.random((10,seqlen,4)))

    for b_idx, b in enumerate(BATCH_SIZES):
        x = np.random.random((b,seqlen,4))

        t = time.time()
        x = tf.constant(x)
        g=[]
        
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            prof, ct = model(x)
            prof = [prof[:,i:i+1] for i in range(seqlen)]

        for i in range(seqlen):
            g.append((x*tape.gradient(prof[i], x)).numpy())
        g.append((x*tape.gradient(ct, x)).numpy())
        times.append(time.time()-t)

        per_100.append((times[-1]/b)*100)
        print("BATCH SIZE: {}\tTIME: {:.2f}\tPER 100: {:.2f}".format(b, times[-1], per_100[-1]))

    print("BEST PER 100: {:.2f}".format(min(per_100)))


------------------
SEQLEN: 1000
BATCH SIZE: 1	TIME: 17.53	PER 100: 1752.96
BATCH SIZE: 32	TIME: 28.40	PER 100: 88.74
BATCH SIZE: 64	TIME: 49.78	PER 100: 77.78
BATCH SIZE: 128	TIME: 101.56	PER 100: 79.35
BEST PER 100: 77.78

------------------
SEQLEN: 2000
BATCH SIZE: 1	TIME: 38.44	PER 100: 3843.96
BATCH SIZE: 32	TIME: 81.94	PER 100: 256.07
BATCH SIZE: 64	TIME: 142.71	PER 100: 222.98
BATCH SIZE: 128	TIME: 286.70	PER 100: 223.99
BEST PER 100: 222.98
